In [ ]:
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
from scipy.stats import sem
from scipy.spatial.distance import squareform, pdist
from keras import backend as K
from keras.models import load_model
import tensorflow as tf


from models import vgg
from models.cvae import CVAE
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [ ]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True, verbose=1)

image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)
yte_rad = np.deg2rad(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

#### Loading pre-trained models

#### Biternion-VGG (Log-likelihood, fixed $\kappa=3$)

In [ ]:
vgg_fixed_kappa = vgg.BiternionVGG(image_height=image_height,
                                   image_width=image_width,
                                   n_channels=3,
                                   predict_kappa=False,
                                   fixed_kappa_value=3.0)

vgg_fixed_kappa.model.load_weights('/Users/sergeyprokudin/BiternionNet/logs/best_models/fixed_kappa/vgg_bit_vm_likelihood_town.best.weights.h5')

In [ ]:
results_fixed = dict()
#results_fixed['train'] = vgg_fixed_kappa.evaluate(xtr, ytr_deg, 'train')
results_fixed['validation'] = vgg_fixed_kappa.evaluate(xval, yval_deg, 'validation')
results_fixed['test'] = vgg_fixed_kappa.evaluate(xte, yte_deg, 'test')

#### Biternion-VGG (Log-likelihood, learned $\kappa$)

In [ ]:
vgg_learned_kappa = vgg.BiternionVGG(image_height=image_height,
                                     image_width=image_width,
                                     n_channels=3,
                                     predict_kappa=True)

vgg_learned_kappa.model.load_weights('/Users/sergeyprokudin/BiternionNet/logs/best_models/learned_kappa/vgg_bit_vm_likelihood_town.best.weights.h5')

In [ ]:
results_learned = dict()
#results_learned['train'] = vgg_learned_kappa.evaluate(xtr, ytr_deg, 'train')
results_learned['validation'] = vgg_learned_kappa.evaluate(xval, yval_deg, 'validation')
results_learned['test'] = vgg_learned_kappa.evaluate(xte, yte_deg, 'test')

#### Biternion-VGG (cosine-loss)

In [ ]:
vgg_cosine = vgg.BiternionVGG(image_height=image_height,
                              image_width=image_width,
                              n_channels=3,
                              predict_kappa=False,
                              fixed_kappa_value=3.0)

vgg_cosine.model.load_weights('/Users/sergeyprokudin/BiternionNet/logs/best_models/cosine/vgg_bit_cosine_town.best.weights.h5')

In [ ]:
results_cosine = dict()
#results_cosine['train'] = vgg_cosine.evaluate(xtr, ytr_deg, 'train')
results_cosine['validation'] = vgg_cosine.evaluate(xval, yval_deg, 'validation')
results_cosine['test'] = vgg_cosine.evaluate(xte, yte_deg, 'test')

#### Biternion-VGG (Von-Mises loss)

In [ ]:
vgg_vm = vgg.BiternionVGG(image_height=image_height,
                          image_width=image_width,
                          n_channels=3,
                          predict_kappa=False,
                          fixed_kappa_value=3.0)

vgg_vm.model.load_weights('/Users/sergeyprokudin/BiternionNet/logs/best_models/von_mises/vgg_bit_von_mises_town.best.weights.h5')

In [ ]:
results_vm = dict()
#results_vm['train'] = vgg_vm.evaluate(xtr, ytr_deg, 'train')
results_vm['validation'] = vgg_vm.evaluate(xval, yval_deg, 'validation')
results_vm['test'] = vgg_vm.evaluate(xte, yte_deg, 'test')

#### CVAE model 

In [ ]:
n_u = 8
cvae_best = CVAE(n_hidden_units=n_u)
cvae_ckpt_path = '/Users/sergeyprokudin/BiternionNet/logs/best_models/cvae/kl_reduction_exepriments//Users/sergeyprokudin/BiternionNet/logs/best_models/cvae/kl_reduction_exepriments/cvae.full_model.overall_best.weights.hdf5'
cvae_best.full_model.load_weights(cvae_ckpt_path)

In [ ]:
results_cvae = dict()
# results_cvae['train'] = cvae_best.evaluate(xtr, ytr_deg, 'train')
results_cvae['validation'] = cvae_best.evaluate(xval, yval_deg, 'validation')
results_cvae['test'] = cvae_best.evaluate(xte, yte_deg, 'test')

In [ ]:
from utils.losses import von_mises_log_likelihood_np

In [ ]:
# cvae_preds = cvae_best.decoder_model.predict(xte)


#### Using Maximum Expected Utility for calculation

In [ ]:
def maximum_expected_utility(ypreds):
    ix = np.argmin(np.sum(squareform(pdist(ypreds, 'euclidean')), axis=1))
    return float(ypreds[ix])

In [ ]:
n_samples = 20

cvae_mu_rad_preds  = np.zeros([n_samples, xte.shape[0], 1])
cvae_kappa_preds  = np.zeros([n_samples, xte.shape[0], 1])
cvae_mu_bit_preds = np.zeros([n_samples, xte.shape[0], 2])

for i in range(0, n_samples):
    cvae_preds = cvae_best.decoder_model.predict(xte)
    cvae_mu_bit_preds[i,:,:] = cvae_preds[:,0:2]
    cvae_mu_rad_preds[i,:,:] = np.deg2rad(bit2deg(cvae_preds[:,0:2])).reshape(-1,1)
    cvae_kappa_preds[i,:,:] = cvae_preds[:,2].reshape(-1,1)

In [ ]:
cvae_mu_rad_preds_mae = [maximum_expected_utility(cvae_mu_rad_preds[:,i,:]) for i in range(0, cvae_mu_rad_preds.shape[1])]
maad_errors = maad_from_deg(np.rad2deg(cvae_mu_rad_preds_mae), yte_deg)
print("MAAD error (test, CVAE-MAE) : %f ± %fSEM" % (np.mean(maad_errors), sem(maad_errors)))

#### Saving predictions from different model

In [ ]:
vm_preds = vgg_vm.model.predict(xte)
vm_preds_mu_rad = np.deg2rad(bit2deg(vm_preds))
vm_preds_kappa = np.ones([xte.shape[0],1])*3
vm_preds_likelihoods = von_mises_log_likelihood_np(yte_bit, vm_preds[:,0:2], learned_preds_kappa)

fixed_preds = vgg_fixed_kappa.model.predict(xte)
fixed_preds_mu_rad = np.deg2rad(bit2deg(fixed_preds))
fixed_preds_kappa = np.ones([xte.shape[0],1])*3
fixed_preds_likelihoods = von_mises_log_likelihood_np(yte_bit, fixed_preds[:,0:2], learned_preds_kappa)

learned_preds = vgg_learned_kappa.model.predict(xte)
learned_preds_mu_rad = np.deg2rad(bit2deg(learned_preds[:,0:2]))
learned_preds_kappa = learned_preds[:,2:]
learned_preds_likelihoods = von_mises_log_likelihood_np(yte_bit, learned_preds[:,0:2], learned_preds_kappa)

In [ ]:
from utils.losses import log_bessel_approx_np

def vm_pdf(vals, mu_rad, kappa):
    return np.exp(kappa * np.cos(vals-mu_rad)) / (2* np.pi * np.exp(log_bessel_approx_np(kappa)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


def plot_vm_dists(fid):
    x_vals = np.arange(0, 2*np.pi, 0.01)
    
    fig, axs = plt.subplots(1, 5, figsize=(25, 5))
    
    axs[0].imshow(xtr[fid])
    
    def _plt_vgg(axis, mu_rad_preds, kappa_preds, method_name):
        axis.set_xticks(([0., .5*np.pi, np.pi, 1.5*np.pi, 2*np.pi]))
        axis.set_xticklabels(["$0$", r"$\frac{\pi}{2}$", r"$\pi$", r"$\frac{3\pi}{2}$", r"$2\pi$"]) 
        axis.set_xlim([-0.001, 2*np.pi+0.001])
        axis.plot(x_vals, np.squeeze(vm_pdf(x_vals, mu_rad_preds[fid], kappa_preds[fid])))
        axis.axvline(yte_rad[fid], color='red')
        axis.set_title("%s ()" % method_name)
        axis.set_ylim([0, 2.0])
        
    def _plt_cvae(axis):
        axis.set_xticks(([0., .5*np.pi, np.pi, 1.5*np.pi, 2*np.pi]))
        axis.set_xticklabels(["$0$", r"$\frac{\pi}{2}$", r"$\pi$", r"$\frac{3\pi}{2}$", r"$2\pi$"]) 
        axis.set_xlim([-0.001, 2*np.pi+0.001])
        for sid in range(0, n_samples):
            axis.plot(x_vals, np.squeeze(vm_pdf(x_vals, cvae_mu_rad_preds[sid, fid], cvae_kappa_preds[sid, fid])))    
        axis.axvline(yte_rad[fid], color='red')
        axis.set_title("CVAE")
        #axis.set_ylim([0, 2.0])
        
    _plt_vgg(axs[1], fixed_preds_mu_rad, fixed_preds_kappa, 'original (Von-Mises loss)')
    _plt_vgg(axs[2], fixed_preds_mu_rad, fixed_preds_kappa, 'fixed kappa')
    _plt_vgg(axs[3], learned_preds_mu_rad, learned_preds_kappa, 'learned kappa')
    _plt_cvae(axs[4])
    fig.suptitle("frame id =%d" %fid)
    #plt.plot((ytr_deg, x2), (y1, y2), 'k-')
    plt.show()

#####  dataset desc http://www.robots.ox.ac.uk/~lav/Papers/benfold_reid_iccv2011/benfold_reid_iccv2011.pdf
#####  https://www.youtube.com/watch?v=eupXTJM_TAw (Demo)

In [ ]:
import time 

for fid in range(0, 20):
    plot_vm_dists(fid)

In [ ]:
cvae_preds_full = cvae_best.full_model.predict([xte, yte_bit])

In [ ]:
mu_prior = cvae_preds_full[:, 0:cvae_best.n_u]
log_sigma_prior = cvae_preds_full[:, cvae_best.n_u:cvae_best.n_u*2]
mu_encoder = cvae_preds_full[:, cvae_best.n_u*2:cvae_best.n_u*3]
log_sigma_encoder = cvae_preds_full[:, cvae_best.n_u*3:cvae_best.n_u*4]
mu_pred = cvae_preds_full[:, cvae_best.n_u*4:cvae_best.n_u*4+2]
kappa_pred = cvae_preds_full[:, cvae_best.n_u*4+2:]

In [ ]:
np.exp(log_sigma_prior[2])

#### The problem is that encoder and prior generate exactly the same distributions for each input

$ p(u|x) = q(u|x,\phi) = p(u)$ 

In [ ]:
mu_prior[0:5]

In [ ]:
mu_encoder[0:5]

#### Generating predictions on random images

In [ ]:
black_img = np.zeros(xte[1].shape)
white_img = np.ones(xte[1].shape)
random_img1 = np.random.rand(xte[1].shape[0], xte[1].shape[1], xte[1].shape[2])
random_img2 = np.random.rand(xte[1].shape[0], xte[1].shape[1], xte[1].shape[2])
random_img3 = np.random.rand(xte[1].shape[0], xte[1].shape[1], xte[1].shape[2])

In [ ]:
cvae_best.full_model.predict

In [ ]:
cvae_best.decoder_model.predict(black_img.reshape([1,50,50,3]))

In [ ]:
def plot_cvae_preds(img, cvae_model, n_samples=10):
    x_vals = np.arange(0, 2*np.pi, 0.01)
    fig, axs = plt.subplots(1, 2, figsize=(7, 3))
    axs[0].imshow(img)
    axs[1].set_xticks(([0., .5*np.pi, np.pi, 1.5*np.pi, 2*np.pi]))
    axs[1].set_xticklabels(["$0$", r"$\frac{\pi}{2}$", r"$\pi$", r"$\frac{3\pi}{2}$", r"$2\pi$"]) 
    axs[1].set_ylim([0, 2.0])
    for i in range(0, n_samples):
        preds = cvae_best.decoder_model.predict(img.reshape([1,50,50,3]))
        mu_preds = np.deg2rad(bit2deg(preds[:, 0:2]))
        kappa_preds = preds[:,2:]
        axs[1].plot(x_vals, np.squeeze(vm_pdf(x_vals, mu_preds[0], kappa_preds[0]))) 
    return

In [ ]:
for i in range(0,5):
    plot_cvae_preds(xte[i], cvae_best, 10)
plot_cvae_preds(black_img, cvae_best, 10)
plot_cvae_preds(white_img, cvae_best, 10)
plot_cvae_preds(random_img1, cvae_best, 10)
plot_cvae_preds(random_img2, cvae_best, 10)
plot_cvae_preds(random_img3, cvae_best, 10)

In [ ]:
loss = cvae_best.full_model.evaluate([xte, yte_bit],yte_bit, verbose=0)